In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, TimeDistributed, LSTM, Dense, Flatten, Dropout
from keras.utils import to_categorical
from keras import metrics
import tensorflow as tf

2025-07-08 11:41:30.338709: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-08 11:41:30.349292: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751964090.360959   30958 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751964090.364459   30958 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751964090.374078   30958 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
# Check whether tensorflow sees the GPU
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
# Load the data
X = np.load("../../mel_spectrograms_train.npy")
y = np.load("../../labels_train.npy")

In [4]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

In [5]:
# === CNN + LSTM Model ===
model = Sequential([
    TimeDistributed(Conv2D(16, (3, 3), activation='relu', padding='same')),
    TimeDistributed(MaxPooling2D((2, 2))),
    TimeDistributed(Conv2D(32, (3, 3), activation='relu', padding='same')),
    TimeDistributed(MaxPooling2D((2, 2))),
    TimeDistributed(Flatten()),
    Dropout(0.1),
    LSTM(64, dropout=0.1, recurrent_dropout=0.1),
    Dropout(0.1),
    Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])

# === Training ===
model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=10, batch_size=16)

model.save('model.keras')

I0000 00:00:1751964092.572999   30958 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5537 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6


Epoch 1/10


I0000 00:00:1751964099.096301   31023 cuda_dnn.cc:529] Loaded cuDNN version 90501


748/748 ━━━━━━━━━━━━━━━━━━━━ 45s 52ms/step - accuracy: 0.7267 - loss: 0.5526 - val_accuracy: 0.7411 - val_loss: 0.5463
Epoch 2/10
748/748 ━━━━━━━━━━━━━━━━━━━━ 34s 45ms/step - accuracy: 0.8372 - loss: 0.3680 - val_accuracy: 0.9428 - val_loss: 0.1590
Epoch 3/10
748/748 ━━━━━━━━━━━━━━━━━━━━ 35s 46ms/step - accuracy: 0.9353 - loss: 0.1742 - val_accuracy: 0.9639 - val_loss: 0.1023
Epoch 4/10
748/748 ━━━━━━━━━━━━━━━━━━━━ 35s 47ms/step - accuracy: 0.9596 - loss: 0.1189 - val_accuracy: 0.9682 - val_loss: 0.0878
Epoch 5/10
748/748 ━━━━━━━━━━━━━━━━━━━━ 36s 48ms/step - accuracy: 0.9649 - loss: 0.1013 - val_accuracy: 0.9766 - val_loss: 0.0616
Epoch 6/10
748/748 ━━━━━━━━━━━━━━━━━━━━ 34s 45ms/step - accuracy: 0.9726 - loss: 0.0783 - val_accuracy: 0.9665 - val_loss: 0.0870
Epoch 7/10
748/748 ━━━━━━━━━━━━━━━━━━━━ 37s 49ms/step - accuracy: 0.9764 - loss: 0.0695 - val_accuracy: 0.9783 - val_loss: 0.0646
Epoch 8/10
748/748 ━━━━━━━━━━━━━━━━━━━━ 36s 48ms/step - accuracy: 0.9827 - loss: 0.0591 - val_accurac